In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import re
import random

In [6]:
total_cost = 0
default_model = "gpt-4o-mini"
verbose = True
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.7, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

main_title = random_word()
if verbose:
    print(main_title)

problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 단어 1개를 제시하라."),
        HumanMessage(content="주제어: " + main_title + " 단어: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
word = word_responses[0][1]
def clean_text(text):
            pattern = r':\s*(.*)$'
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
            text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
            return text.strip()
word = clean_text(word)
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_10'
random_question = find_random_question(keyword, problem_type, detail_type=1, count=2, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)
    
example_dict = []
for item in random_question:
    example_text = item['example']
    problem_generate_model.request_models_responses([
    SystemMessage(content="문장의 주제 단어 1개를 제시하라."),
    HumanMessage(content="문장: "+ example_text + "주제 단어: "),
        ]
    )
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_generate_model.get_model_responses()[0][2]).group(1))
    title_responses = problem_generate_model.get_model_responses()[0][1]
    temp = {"주제": title_responses, "예문": example_text, "선택지": ', '.join(item['selector']), "정답": item['answer']}
    example_dict.append(temp)
if verbose:
    print(example_dict)
example_prompt = ""
for item in example_dict:
    example_str = f"주제: {item['주제']}, 예문: {item['예문']}\n 선택지: {item['선택지']}\n 정답: {item['정답']}"
    example_prompt += example_str + "\n"
if verbose:
    print(example_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content="#너는 문제를 출제하는 선생님이야. 예시처럼 주어지는 주제에 대한 글의 올바른 내용 고르기 문제 1개를 생성해줘. 정답은 1개로 명확한 선택지를 구성해야해. #답변 예시 "
            + example_prompt
        ),
        HumanMessage(content="주제: " + word),
    ]
)
problem_responses = problem_generate_model.get_model_responses()
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_responses[0][2]).group(1))
if verbose:
    print(problem_responses[0][1])
    
def extract_parts(text):
    # 예문 추출
    example_pattern = r'예문:\s*(.*)'
    example_match = re.search(example_pattern, text)
    example = example_match.group(1).strip() if example_match else None

    # 선택지 추출
    options_pattern = r'선택지:\s*(.*)'
    options_match = re.search(options_pattern, text)
    if options_match:
        options = options_match.group(1).strip().split(', ')
        for option in options:
            option = clean_text(option)
    else:
        options = []

    # 정답 추출
    answer_pattern = r'정답:\s*(.*)'
    answer_match = re.search(answer_pattern, text)
    answer = answer_match.group(1).strip() if answer_match else None

    return example, options, answer

example, options, answer = extract_parts(problem_responses[0][1])
random.shuffle(options)
result = {
    "example": example,
    "selector": options,
    "answer": answer,
    "eval_answer": answer,
    "eval_explain": "",
    "total_cost": total_cost
}
print(result)

경찰서
gpt-4o-mini | 
경찰관

OpenAI Tokens Used: 44
    Prompt Tokens: 41
    Completion Tokens: 3
Successful Requests: 1
Total Cost (USD): $3e-05
Total Cost (KRW): ₩0.0405
--------------------------------------------------------------------------------------------------
[{'example': '문화와 역사적 가치가 높지만 국가에서 관리하지 못하고 있는 토지,자연, 건물 등이 많이 있다.이를 관리하고 보존하려는 시민운동이 새롭게 시작 되고 있다.사라질 위기에 처해 있는 중요한 자산을 지키기 위해 시민들이 자발적으로 모금에 나선 것이다.이러한 시민의 노력으로 최근 희귀식물인 ‘매화마름’이 보존되는 등 가시적인 성과가 나타나고 있다.이러한 운동이 앞으로도 지속되기 위해서는 시민들의 적극적인 동참이 필요하다.', 'selector': ['시민들은 국가의 지원을 받아 모금 활동을 했다.', '시민들이 국가 토지 관리에 적극 참여하고 있다.', '시민들이 역사적 가치가 높은 자산을 보존하기 시작했다.', '시민운동이 희귀식물 보존에까지는 미치지 못하고 있다.'], 'answer': '시민들이 역사적 가치가 높은 자산을 보존하기 시작했다.'}, {'example': '동물원의 동물들은 빠른 번식으로 인해 개체 수 조절이 어렵다.그러다 보면 한정된 공간에 너무 많은 동물들이 함께 있게 되어 스트레스를 많이 받게 된다.그래서 서울의 한 동물원에서는 사자의 개체 수를 줄이기 위해 많은 수의 사자를 다른 동물원에 보내고 대신 개체 수가 적은 낙타를 들여 오는 방법을 썼다.그 결과 개체 수가 줄어든 사자들은 스트레스를 덜 받게 되었다.이러한 동물원 간 교류가 동물의 서식 환경 개선을 위한 하나의 방안이 될 수 있을 것이다.', 'selector': ['동물원의 동물들은 번식으로